# Estimating Motors from a Variety of Geometric Data in 3D Conformal Geometric Algebra

Robert Valkenburg and Leo Dorst

In [1]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=2, suppress=True)
import versor as vsr
from versor.drawing import *
from game import (MotorEstimationSolver, VDMotorEstimationSolver)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import versor as vsr

## Dataset Generation

In [3]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]


In [4]:
n_points = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.rand(3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

In [5]:
lines_a = [vsr.Dll(vsr.Vec(*np.random.normal(0.0,0.8,3)).null(),
                   vsr.Vec(*np.random.normal(0.0,0.8,3)).null())
          for i in range(10)]
lines_b = [line.spin(motor) for line in lines_a]

In [6]:
n_planes = 100
planes_a = []
for i in range(10):
    dir_vec = vsr.Vec(*np.random.normal(0.0, 0.8, 3)).unit()
    distance = np.random.normal(0.0, 0.8,1)
    planes_a.append(vsr.Dlp(dir_vec[0],dir_vec[1],dir_vec[2],distance[0]))
planes_b = [plane.spin(motor) for plane in planes_a]

In [7]:
n_drvs = 10
drvs_a = [vsr.Vec(*np.random.rand(3)).unit().drv() for i in range(10)]
drvs_b = [d.spin(motor) for d in drvs_a]

In [8]:
vd = VDMotorEstimationSolver()
vd.add_dual_plane_observations(planes_a, planes_b)
# vd.add_dual_line_observations(lines_a, lines_b)
# vd.add_point_observations(points_a, points_b_noisy)
vd_estimated_motor = vd.solve()
print(np.array(vd_estimated_motor))

[-0.87 -0.    0.5   0.    0.    0.    0.    0.  ]


In [9]:
for a, b in zip(points_a, points_b):
    print(b - a)

Dlp: [ 0.8361 1 1.004 1.569 ]
Dlp: [ 0.002618 1 0.9409 2.229 ]
Dlp: [ 0.296 1 1.391 2.486 ]
Dlp: [ 0.3923 1 1.513 3.34 ]
Dlp: [ 0.4327 1 0.9013 2.328 ]
Dlp: [ 0.2311 1 0.5613 1.815 ]
Dlp: [ 0.6047 1 0.8395 1.713 ]
Dlp: [ -0.0409 1 1.429 2.757 ]
Dlp: [ 0.1448 1 0.5395 1.328 ]
Dlp: [ 0.644 1 1.529 2.911 ]


In [10]:
scene = Scene(children=[point_mesh(points_a[0],color='hotpink'),
                        point_mesh(points_b[0],color='hotpink'),
#                         plane_mesh(points_b[0] - points_a[0], width=10, height=10,color='lightskyblue'),
                       frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

In [55]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
# for a,b in zip(drvs_a, drvs_b):
#     mes.add_direction_vector_inner_product_residual_block(a,b)
for a,b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(a,b)
#     mes.add_line_commutator_residual_block(a,b) 
#     mes.add_line_inner_product_residual_block(a,b)
#     mes.add_line_angle_distance_norm_residual_block(a,b)
    
# for a,b in zip(points_a, points_b):
#     mes.add_point_distance_residual_block(a,b)
#     mes.add_point_correspondences_residual_block(a,b)
# for a, b, c, d, e, f in zip(points_a, points_b, lines_a, lines_b, planes_a, planes_b):
#     mes.add_point_difference_residual_block(a,b)
#     mes.add_line_correspondences_residual_block(c,d)
#     mes.add_dual_plane_difference_residual_block(e,f)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
# mes.num_linear_solver_threads = 4
# mes.num_threads = 4
# mes.function_tolerance  = 1e-16

mes.max_num_iterations = 1000
# mes.minimizer_type = 'LINE_SEARCH'

te_estimated_motor, summary, motors = mes.solve()
print(summary['full_report'])
print(np.array(te_estimated_motor))


Solver Summary (v 1.11.0-eigen-(3.2.92)-lapack-suitesparse-(4.4.6)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            10                       10
Residual                                   60                       60

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1

Cost:
Initial                          1.218291e+02
Final              

In [37]:
print(np.array(motor))

[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]


In [13]:
np.allclose(vd_estimated_motor, te_estimated_motor, atol=1e-03)

False

In [14]:
dlp = vsr.Dlp(1,1,1,5)
Jac = np.zeros((5,8))
for i in range(8):
    ei = vsr.Mot(0,0,0,0,0,0,0,0)
    ei[i] = 1.
    Jac[:,i] = np.array((vsr.CGA(motor) * vsr.CGA(dlp) * vsr.CGA(ei.rev()) +
                         vsr.CGA(ei) * vsr.CGA(dlp) * vsr.CGA(motor.rev())))[1:6]
Jac = np.delete(Jac,3,0)
print(Jac)

[[  0.73   1.73   2.73   1.     0.     0.     0.     0.  ]
 [  1.73  -0.73  -1.     2.73   0.     0.     0.     0.  ]
 [  2.73   1.    -0.73  -1.73   0.     0.     0.     0.  ]
 [ 11.26   1.    -4.5    1.    -0.73  -1.73  -2.73  -1.  ]]


In [15]:
Jac3 = np.zeros((8,6))
Jac3[:,0] = vsr.Dll(1,0,0,0,0,0) * motor
Jac3[:,1] = vsr.Dll(0,1,0,0,0,0) * motor
Jac3[:,2] = vsr.Dll(0,0,1,0,0,0) * motor
Jac3[:,3] = vsr.Dll(0,0,0,1,0,0) * motor
Jac3[:,4] = vsr.Dll(0,0,0,0,1,0) * motor
Jac3[:,5] = vsr.Dll(0,0,0,0,0,1) * motor
print(Jac3)

[[ 0.    0.5   0.    0.    0.    0.  ]
 [ 0.87  0.   -0.5   0.    0.    0.  ]
 [ 0.    0.87  0.    0.    0.    0.  ]
 [ 0.5   0.    0.87  0.    0.    0.  ]
 [-0.43 -0.18  0.25  0.87  0.    0.5 ]
 [ 0.68 -0.25 -0.18  0.    0.87  0.  ]
 [ 0.25  0.68  0.43 -0.5   0.    0.87]
 [-0.18  0.43 -0.68  0.    0.5   0.  ]]


In [16]:
np.dot(Jac,Jac3)

array([[ 2.  ,  2.73,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.73,  0.  ,  2.73,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.73, -2.  ,  0.  ,  0.  ,  0.  ],
       [-0.  ,  0.  ,  0.  ,  0.73, -2.  , -2.73]])

In [17]:
p1s = [vsr.Vec(1,0,0).null(), vsr.Vec(0,1,0).null(), vsr.Vec(-1,0,1).null()] 
p2s = [p1i.spin(motor) for p1i in p1s] 
p3s = [(p2i.vec() - p1i.vec()).null() for p1i, p2i in zip(p1s,p2s)]
c1 = vsr.Cir(*p1s)
c2 = vsr.Cir(*p2s)
c3 = vsr.Cir(*p3s)

In [18]:
np.array((vsr.CGA(motor) * vsr.CGA(c1) * vsr.CGA(vsr.Mot(1,0,0,0,0,0,0,0).rev()) +
 vsr.CGA(vsr.Mot(1,0,0,0,0,0,0,0)) * vsr.CGA(c1) * vsr.CGA(motor.rev())))[16:26]

array([ 4.7 ,  4.46, -1.73, -0.27,  4.58, -1.8 ,  0.3 , -1.  ,  1.  , -1.  ])

In [19]:
JacCircle = np.zeros((10,8))
for i in range(8):
    ei = vsr.Mot(0,0,0,0,0,0,0,0)
    ei[i] = 1.
    JacCircle[:,i] = np.array((vsr.CGA(motor) * vsr.CGA(c1) * vsr.CGA(ei.rev()) +
                               vsr.CGA(ei) * vsr.CGA(c1) * vsr.CGA(motor.rev())))[16:26]
print(JacCircle)

[[ 4.7   1.5   0.87  1.87  0.27 -1.73 -4.46 -1.  ]
 [ 4.46  1.    0.27 -1.73  0.    0.    0.    0.  ]
 [-1.73 -0.27  1.   -4.46  0.    0.    0.    0.  ]
 [-0.27  1.73  4.46  1.    0.    0.    0.    0.  ]
 [ 4.58  1.25 -0.3  -1.8   1.87  2.87 -7.16 -2.5 ]
 [-1.8   0.3   1.25 -4.58 -2.5   7.16  2.87 -1.87]
 [ 0.3   1.8   4.58  1.25 -7.16 -2.5  -1.87 -2.87]
 [-1.    1.   -1.    2.46 -1.    4.46 -1.73 -0.27]
 [ 1.    1.   -2.46 -1.   -4.46 -1.   -0.27  1.73]
 [-1.    2.46  1.   -1.    1.73  0.27 -1.    4.46]]


In [20]:
np.dot(JacCircle,Jac3)

array([[-0.  , -0.  ,  0.  ,  2.46, -2.  , -3.73],
       [ 0.  ,  2.46, -2.  ,  0.  ,  0.  ,  0.  ],
       [-2.46,  0.  , -3.73,  0.  ,  0.  ,  0.  ],
       [ 2.  ,  3.73,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -5.  , -3.63,  5.2 ,  1.23, -5.27],
       [ 5.  , -0.  , -3.54, -3.6 ,  5.27,  1.23],
       [ 3.63,  3.54, -0.  , -5.27, -3.6 , -5.2 ],
       [ 5.2 , -3.6 , -0.  ,  0.  ,  3.73, -2.  ],
       [ 1.23,  0.  , -3.6 , -3.73,  0.  , -2.46],
       [-0.  ,  1.23, -5.2 ,  2.  ,  2.46,  0.  ]])

In [21]:
trs = vsr.Vec(1,2,3).trs() * vsr.Biv(0,pi/6,0).exp()
print(trs)
cdiff = c1.spin(trs)

Mot: [ 0.87 0 -0.5 0 -1.2 -0.87 -1 -0.5 ]


In [22]:
ans = c1.dual().comm(cdiff.dual()).undual()
scene = Scene(children=[circle_mesh(c1,color='hotpink'),
                        point_mesh(c1.pnt(),color='hotpink'),
                        circle_mesh(cdiff,color='lightskyblue'),
                        point_mesh(cdiff.pnt(),color='lightskyblue'),
                        line_mesh(vsr.Dll(c1.pnt(), cdiff.pnt())),
                        bivector_mesh(cdiff.dir().unit()),
                        circle_mesh(ans,color='green'),
                        point_mesh(ans.pnt(),color='green'),
#                         vector_mesh(ans.dual().dir().vec(), arrow=True, position=(ans).pnt()),
                        frame_mesh(),
                        AmbientLight(color='#cccccc')] +\
             [point_mesh(p, color='red') for p in ans.dual().split()]
             )

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

In [23]:
c1.pnt()


Pnt: [ 0 0 0.5 1 0.125 ]

In [24]:
cdiff.pnt()

Pnt: [ 0.567 2 3.25 1 7.442 ]

In [25]:
ans.pnt()

Pnt: [ 0.04788 0.8923 1.848 1 2.108 ]

In [26]:
ans = c1.dual().comm(cdiff.dual())
ans

Par: [ 1.254 12.67 3.663 5.49 -7.5 -18.29 -5.826 -0.5849 13.19 40.25 ]

In [27]:
ans.radius()

1.4672999095746644

In [28]:
ans.dir().unit().unduale()

TypeError: Unable to convert function return value to a Python type! The signature was
	(Drv) -> vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)19, (short)21, (short)22> >

In [ ]:
cdiff.dir().unit().unduale() 

In [ ]:
c1.dir().unit().unduale() 

In [ ]:
c1.radius()

In [ ]:
cdiff.radius()

In [ ]:
c1.pnt().vec() - cdiff.pnt().vec()

In [ ]:
(c1 - cdiff).dual().dir().vec()

In [ ]:
ans[0]

In [ ]:
- np.outer(l1,l2)ans[1].undual()

In [ ]:
vsr.Lin(c1 - cdiff).__print_debug_info_console()

In [ ]:
c1.__print_debug_info_console()
cdiff.__print_debug_info_console()

In [ ]:
dlp2 = dlp.spin(motor)
pd = dlp2 - dlp
scene = Scene(children=[
        vector_mesh(dlp.dir().vec(),color='hotpink', arrow=True),
        plane_mesh(dlp,color='hotpink'),
        plane_mesh(dlp2,color='lightskyblue'),
        plane_mesh(dlp2-dlp,color='red'),
                        frame_mesh(),
                        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
display(renderer)

In [ ]:
dlp2.dir()

In [ ]:
dlp

In [ ]:
pd

In [ ]:
l1 = lines_a[0]
l2 = lines_b[0]
Z = np.outer(l2,l1) - np.outer(l1,l2)
print(Z)

In [ ]:
np.linalg.eig(Z)

In [ ]:
np.linalg.norm(l2.comm(l1) * 2.) / np.sqrt(2)

In [ ]:
Z = np.zeros(6)
for a,b in zip(lines_a, lines_b):
    Z = Z + np.outer(a,b)

In [ ]:
U,s,Vt = np.linalg.svd(Z)
np.dot(U.T,Vt)

In [ ]:
np.array(motor)

In [ ]:
n_circles = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.3, 3)).null() 
            for i in range(n_circles)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

radiuss = [np.random.rand() for i in range(n_circles)]
bivs = [vsr.Vec(*np.random.normal(0.0, 0.3, 3)).unit().unduale()
            for i in range(n_circles)]

circs_a = [vsr.Cir(p,r,b) for p,r,b in zip(points_a, radiuss, bivs)]
# circs_b = [vsr.Cir(p,r,b) for p,r,b in zip(points_b_noisy, radiuss, bivs)]
circs_b = [circle.spin(motor) for circle in circs_a]


In [ ]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# initial_motor = vsr.Vec(2,3,4).trs() * vsr.Rot(vsr.Biv(0,-1,0) * np.pi/6)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(circs_a, circs_b):
    a1,a2 = a.dual().split()
    b1,b2 = b.dual().split()
    mes.add_line_correspondences_residual_block(vsr.Dll(a1,a2), vsr.Dll(b1,b2))
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_motor, summary, motors) = mes.solve()
# print(summary['full_report'])
print(estimated_motor)

In [ ]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# initial_motor = vsr.Vec(2,3,4).trs() * vsr.Rot(vsr.Biv(0,-1,0) * np.pi/6)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(circs_a, circs_b):
#     mes.add_circle_difference_residual_block(a,b)
    mes.add_circle_commutator_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_motor, summary, motors2) = mes.solve()
print(summary['full_report'])
print(estimated_motor)

In [ ]:
[np.allclose(m, m2,atol=1e-05) for m,m2 in zip(motors, motors2)]

In [ ]:
print(zip(motors, motors2)[-1])

In [ ]:
scene = Scene(children=
#               [circle_mesh(c,color='hotpink') for c in circs_a] +\
#               [circle_mesh(c,color='lightskyblue') for c in circs_b] +\
              [circle_mesh(c.spin(motors[0]), color='red') for c in circs_a] +\
              [circle_mesh(c.spin(motors[1]), color='yellow') for c in circs_a] +\
              [circle_mesh(c.spin(motors2[1]), color='magenta') for c in circs_a] +\

#               [circle_mesh(c.spin(motors[2]), color='green') for c in circs_a] +\
# #               [circle_mesh(c.spin(motors[3]), color='cyan') for c in circs_a] +\
#               [circle_mesh(c.spin(motors[4]), color='blue') for c in circs_a] +\

#               [circle_mesh(c.spin(motors[-4]), color='green') for c in circs_a] +\
#               [circle_mesh(c.spin(motors2[-4]), color='red') for c in circs_a] +\
              [
        
                        frame_mesh(),
                        AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white')
renderer.width = '1024'
renderer.height = '768'
display(renderer)